In [16]:
# So, basically, the Black-Sholes model is a estimates the theoretical value of a given derivative
# based on explicit, discrete, external instruments. It (BSM) considers the impact of time until
# expiration as well as other risk factors. 

# It is set up as a differential equation with respect to an arbirtary number of variables, each of which
# increase the complexity linearly. Only used to consider European-style options, as it does not consider
# the fact that American options can be exercised before the expiration date occurs.

# The model posits the idea that instruments are distributed log-normally, which basically just means that
# they are the sort of normal in the positive x direction and positive y direction. This is because 
# the option is essentially risk-free apart from the price of the option initially, which is sunk after
# purchase. It also assumes they follow a random walk (stochastic process) with a constant drift and a 
# constant volatility. It is set up to predict the price of a CALL OPTION. There is an alternative equation
# that can be derived closed-form that predicts the price of a PUT OPTION. Other models of this nature include
# the binomial pricing model or the Monte-Carlo model.

# Assumptions:
        # 1. No dividends are paid out during the life of the option.
        # 2. Markets are entirely random.
        # 3. No transaction costs when purchasing the option (frictionless)
        # 4. The risk-free rate and volatility are known and constant
        # 5. The returns of the actual underlying assets are NORMALLY distributed (hence the lognormal of the option)
        # 6. Option is European.

# The base BSM is given as: C =  (Current underlying price)(Normal distribution of d1) - (Strike price)(exp(-risk free rate * time to expiration))(Normal distribution d2)
# where d1 = (ln(current price / strike price) + (risk-free + ))

In [17]:
# Black sholes calculator
from typing import Callable
from typing import List
import yfinance as yf
import numpy as np
import scipy.stats as si
import matplotlib.pyplot as plt
import mplfinance as mpf
import plotly.graph_objects as go
from datetime import datetime
import pandas as pd

class BlackSholesModel:
    def __init__(self, S, K, T, r, sigma):
        self.S = S
        self.K = K
        self.T = T
        self.r = r
        self.sigma = sigma

    def d1(self):
        return (np.log(self.S / self.K) + (self.r + (0.5 * (self.sigma ** 2)))) / (self.sigma * np.sqrt(self.T))
    
    def d2(self):
        return self.d1() - (self.sigma * np.sqrt(self.T))
    
    def call_option_price(self):
        return (self.S * si.norm.cdf(self.d1(), 0, 1)) - (self.K * np.exp(-1 * self.r * self.T) * si.norm.cdf(self.d2(), 0, 1))
    
    def put_option_price(self):
        return (self.K * np.exp(-self.r * self.T) * si.norm.cdf(-self.d2(), 0, 1) - self.S * si.norm.cdf(-self.d1(), 0, 1))
    
    # Example usage

bsm_example = BlackSholesModel(S=100, K=100, T=1, r=0.05, sigma=0.2)
print(f"Call Option Price: {bsm_example.call_option_price()}")
print(f"Put Option Price: {bsm_example.put_option_price()}")



Call Option Price: 10.450583572185565
Put Option Price: 5.573526022256971


In [29]:
# Historical volatlity: the HV of a stock is basically a measure of the distribution of returns, or the variability in
# return magnitude, of something over a period. You essentially want to find the average deviation the average price.
# Computed by making use of the standard deviation. 

# Collect the historical prices for the asset.
# Compute the expected price (mean) of the historical prices.
# Work out the difference between the average price and each price in the series.
# Square the differences from the previous step.
# Determine the sum of the squared differences.
# Divide the differences by the total number of prices (find variance).
# Compute the square root of the variance computed in the previous step.

class Stock:
    def __init__(self, ticker, interval, start, end):
        self.interval = interval
        self.start = start
        self.end = end
        self.ticker = ticker

    def get_data(self):
        self.data = yf.download(self.ticker, self.start, self.end, self.interval)
        if isinstance(self.data.columns, pd.MultiIndex):
            self.data.columns = self.data.columns.get_level_values(0)
        # self.data.reset_index(names='Date', inplace = True)
        self.close = self.data['Close']

def hv(ticker: str, interval: int = 4, start: str = '2023-01-01', end = datetime.now()) -> float:
    stock = Stock(ticker, interval, start, end)
    stock.get_data()
    print(stock.data)

hv('aapl', 1)

[*********************100%***********************]  1 of 1 completed

1 Failed download:
['AAPL']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))


Empty DataFrame
Columns: [Adj Close, Close, High, Low, Open, Volume]
Index: []
